## Carregar o arquivo XML em um Pandas DataFrame

In [1]:
import json
import pandas as pd
import xmltodict

with open('../data/sources/2017.xml') as file:
    contents = file.read()
contents = xmltodict.parse(contents)

propositions = [pd.Series(dict(x)) for x in contents['proposicoes']['proposicao']]
propositions = pd.DataFrame(propositions)

In [2]:
dict_cols = [
    'apreciacao',
    'autor1',
    'orgaoNumerador',
    'regime',
    'situacao',
    'tipoProposicao',
    'ultimoDespacho',
]

def dict_to_multiple_series(row, cols):
    for col, value in row[cols].items():
        series = pd.Series(dict(value))
        series.index = ['{}_{}'.format(col, x) for x in series.index]
        row = row.append(series)
    return row.drop(cols)

propositions = propositions \
    .apply(lambda row: dict_to_multiple_series(row, cols=dict_cols), axis=1)

dict_cols = [
    'situacao_orgao',
    'situacao_principal',
]
propositions = propositions \
    .apply(lambda row: dict_to_multiple_series(row, cols=dict_cols), axis=1)

print(propositions.shape)
propositions.head()

(2461, 33)


,ano,datApresentacao,id,indGenero,nome,numero,qtdAutores,qtdOrgaosComEstado,txtEmenta,txtExplicacaoEmenta,...,situacao_id,tipoProposicao_id,tipoProposicao_nome,tipoProposicao_sigla,ultimoDespacho_datDespacho,ultimoDespacho_txtDespacho,situacao_orgao_codOrgaoEstado,situacao_orgao_siglaOrgaoEstado,situacao_principal_codProposicaoPrincipal,situacao_principal_proposicaoPrincipal
0,2017,30/11/2017 19:11:00,2164147,o,PL 9248/2017,9248,1,2,"Dispõe sobre o acolhimento, pelo Banco Central...",None,...,None,139,Projeto de Lei,PL,1/12/2017 16:05:00,"Às Comissões de Trabalho, de Administração e S...",186,Diversos,0,None
1,2017,30/11/2017 16:27:00,2164134,o,PL 9247/2017,9247,1,1,Autoriza o Conselho Curador do Fundo de Garant...,None,...,905,139,Projeto de Lei,PL,None,None,100001,SECAP(SGM),0,None
2,2017,30/11/2017 16:00:00,2164131,o,PL 9246/2017,9246,1,1,"Altera a Lei n° 8.742, de 7 de dezembro de 199...",None,...,905,139,Projeto de Lei,PL,None,None,100001,SECAP(SGM),0,None
3,2017,30/11/2017 15:23:00,2164120,o,PL 9245/2017,9245,1,1,Dispõe sobre a obrigatoriedade de as empresas ...,None,...,905,139,Projeto de Lei,PL,None,None,100001,SECAP(SGM),0,None
4,2017,30/11/2017 13:25:00,2164073,o,PL 9244/2017,9244,1,1,"Altera a Lei nº 9.610, de 19 de fevereiro de 1...",None,...,905,139,Projeto de Lei,PL,None,None,100001,SECAP(SGM),0,None


In [3]:
category_cols = [
    'autor1_codPartido',
    'autor1_idecadastro',
    'autor1_txtNomeAutor',
    'autor1_txtSiglaPartido',
    'autor1_txtSiglaUF',
    'orgaoNumerador_id',
    'orgaoNumerador_nome',
    'orgaoNumerador_sigla',
    'regime_codRegime',
    'regime_txtRegime',
    'situacao_descricao',
    'situacao_id',
    'tipoProposicao_id',
    'tipoProposicao_nome',
    'tipoProposicao_sigla',
    'situacao_orgao_codOrgaoEstado',
    'situacao_orgao_siglaOrgaoEstado',
    'situacao_principal_codProposicaoPrincipal',
    'situacao_principal_proposicaoPrincipal',
]
for col in category_cols:
    propositions[col] = propositions[col].astype('category')

In [4]:
propositions.iloc[0]

ano                                                                                       2017
datApresentacao                                                            30/11/2017 19:11:00
id                                                                                     2164147
indGenero                                                                                    o
nome                                                                              PL 9248/2017
numero                                                                                    9248
qtdAutores                                                                                   1
qtdOrgaosComEstado                                                                           2
txtEmenta                                    Dispõe sobre o acolhimento, pelo Banco Central...
txtExplicacaoEmenta                                                                       None
apreciacao_id                                     

In [5]:
propositions['autor1_txtNomeAutor'].value_counts().head(20)

Rômulo Gouveia                 76
Francisco Floriano             70
Heuler Cruvinel                57
Capitão Augusto                53
Aureo                          52
Carlos Bezerra                 44
Cabo Sabino                    43
Veneziano Vital do Rêgo        39
Professor Victório Galli       35
Laura Carneiro                 35
Augusto Carvalho               34
Carlos Henrique Gaguim         29
Vitor Valim                    29
Rubens Pereira Júnior          22
Gorete Pereira                 21
Antonio Carlos Mendes Thame    21
Mariana Carvalho               21
Marco Maia                     20
Laudivio Carvalho              20
Roberto Sales                  20
Name: autor1_txtNomeAutor, dtype: int64

In [6]:
propositions['situacao_descricao'].value_counts()

Tramitando em Conjunto                                       1186
Aguardando Parecer                                            475
Aguardando Designação de Relator                              319
Pronta para Pauta                                              72
Aguardando Despacho do Presidente da Câmara dos Deputados      59
Aguardando Deliberação                                         58
Retirado pelo Autor                                            42
Arquivada                                                      24
Prioridade (Art. 151, II, RICD)                                15
Devolvida ao Autor                                             14
Urgência (Art. 155, RICD)                                      12
Aguardando Definição Encaminhamento                             7
Transformado em Norma Jurídica                                  5
Aguardando Apreciação pelo Senado Federal                       4
Aguardando Deliberação de Recurso                               2
Aguardando

## Calcular palavras-chave das ementas

In [7]:
import nltk
import string

nltk.download('stopwords', quiet=True)
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords = stopwords + list(string.punctuation)

import unicodedata

nltk.download('punkt', quiet=True)

def normalize_string(string):
    if isinstance(string, str):
        nfkd_form = unicodedata.normalize('NFKD', string.lower())
        return nfkd_form.encode('ASCII', 'ignore').decode('utf-8')

def words(val):
    text = normalize_string(val)
    if text:
        text = [word for word in nltk.word_tokenize(text) if word not in stopwords]
        return ' '.join(text)

propositions['tokenized_ementa'] = propositions['txtEmenta'].apply(words)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = propositions['tokenized_ementa']
model = TfidfVectorizer(stop_words=stopwords)
tf_idf = model.fit_transform(X)

In [9]:
def lookup_keyword(row):
    tf_idf_row = tf_idf[row['index']].toarray().ravel()
    keywords = {keyword: value
                for keyword, value in zip(model.get_feature_names(), tf_idf_row)
                if value}
    row['keyword'] = max(keywords.items(), key=lambda x: x[1])[0]
    return row

propositions = propositions.reset_index().apply(lookup_keyword, axis=1)

In [10]:
propositions.head()

,index,ano,datApresentacao,id,indGenero,nome,numero,qtdAutores,qtdOrgaosComEstado,txtEmenta,...,tipoProposicao_nome,tipoProposicao_sigla,ultimoDespacho_datDespacho,ultimoDespacho_txtDespacho,situacao_orgao_codOrgaoEstado,situacao_orgao_siglaOrgaoEstado,situacao_principal_codProposicaoPrincipal,situacao_principal_proposicaoPrincipal,tokenized_ementa,keyword
0,0,2017,30/11/2017 19:11:00,2164147,o,PL 9248/2017,9248,1,2,"Dispõe sobre o acolhimento, pelo Banco Central...",...,Projeto de Lei,PL,1/12/2017 16:05:00,"Às Comissões de Trabalho, de Administração e S...",186,Diversos,0,NaN,dispoe sobre acolhimento banco central brasil ...,voluntarios
1,1,2017,30/11/2017 16:27:00,2164134,o,PL 9247/2017,9247,1,1,Autoriza o Conselho Curador do Fundo de Garant...,...,Projeto de Lei,PL,None,None,100001,SECAP(SGM),0,NaN,autoriza conselho curador fundo garantia tempo...,conselho
2,2,2017,30/11/2017 16:00:00,2164131,o,PL 9246/2017,9246,1,1,"Altera a Lei n° 8.742, de 7 de dezembro de 199...",...,Projeto de Lei,PL,None,None,100001,SECAP(SGM),0,NaN,altera lei n 8.742 7 dezembro 1993 dispoe sobr...,742
3,3,2017,30/11/2017 15:23:00,2164120,o,PL 9245/2017,9245,1,1,Dispõe sobre a obrigatoriedade de as empresas ...,...,Projeto de Lei,PL,None,None,100001,SECAP(SGM),0,NaN,dispoe sobre obrigatoriedade empresas forneced...,mesmos
4,4,2017,30/11/2017 13:25:00,2164073,o,PL 9244/2017,9244,1,1,"Altera a Lei nº 9.610, de 19 de fevereiro de 1...",...,Projeto de Lei,PL,None,None,100001,SECAP(SGM),0,NaN,altera lei 9.610 19 fevereiro 1998 dispor sobr...,aparencia


In [11]:
propositions \
    .groupby(['keyword']) \
    .size() \
    .sort_values(ascending=False) \
    .head(20)

keyword
848             22
saude           15
dia             14
educacao        13
trabalho        10
transito        10
armas            9
adolescente      8
crimes           8
consumidor       8
inciso           7
idoso            7
programa         6
publicos         6
anexo            5
nao              5
eleitoral        5
combustiveis     5
1984             4
judicial         4
dtype: int64